In [1]:
# This command forces an upgrade of the two packages with conflicts.
# Pip will look at all other installed packages and find compatible versions.
!pip install --upgrade numpy protobuf

  Using cached numpy-2.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
Using cached numpy-2.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.21 requires opencv-contrib-python, which is not installed.
mediapipe 0.10.21 requires numpy<2, but you have numpy 2.3.3 which is incompatible.
mediapipe 0.10.21 requires protobuf<5,>=4.25.3, but you have protobuf 6.32.1 which is incompatible.
tensorflow 2.19.1 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.3 which is incompatible.
tensorflow 2.19.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.32.1 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.3 which is incompatible.
numba 0.60.0 re

In [2]:
# -----------------------------------------------------------------
# FINAL CORRECTED SCRIPT FOR INSTALLING ALL DEPENDENCIES
# -----------------------------------------------------------------

# Step 1: Install the TensorFlow, MediaPipe, and Computer Vision stack.
# We install them together to let pip resolve the dependencies for a compatible set.
print("📦 Installing TensorFlow, MediaPipe, OpenCV, and Scikit-learn...")
# Step 1: Use Numpy 1.26.x (compatible with TF 2.x & Mediapipe)
!pip install numpy==1.26.4 --quiet

# Step 2: Install TensorFlow, Mediapipe, OpenCV, and Scikit-learn
!pip install tensorflow==2.15.0 mediapipe==0.10.14 opencv-contrib-python==4.8.1.78 scikit-learn==1.5.2 matplotlib --quiet

# Step 2: Install the latest compatible PyTorch and PyTorchVideo.
# By removing the version number, pip will automatically select the newest version
# that works with your Colab environment.
print("📦 Installing PyTorch and PyTorchVideo...")
!pip install -q torch torchvision --extra-index-url https://download.pytorch.org/whl/cu118
!pip install -q "git+https://github.com/facebookresearch/pytorchvideo.git"
!pip install -q decord

print("\n✅ All installations complete!")

📦 Installing TensorFlow, MediaPipe, OpenCV, and Scikit-learn...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.21 requires opencv-contrib-python, which is not installed.
mediapipe 0.10.21 requires protobuf<5,>=4.25.3, but you have protobuf 6.32.1 which is incompatible.
tensorflow 2.19.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.32.1 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.19.1 which is incompatible.
ERROR: Could not find a version that satisfies the requirement tensorflow==2.15.0 (from versi

In [3]:
from google.colab import drive
# --- Mount Google Drive ---
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# -*- coding: utf-8 -*-
# SCRIPT 2: MEDIAPIPE + LSTM TRAINING

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
from google.colab import drive


# --- Parameters ---
DATA_PATH = '/content/drive/MyDrive/action_data_processed'
MODEL_SAVE_PATH = '/content/drive/MyDrive/mediapipe_lstm_model.h5'
SEQUENCE_LENGTH = 30
NUM_FEATURES = 225 # (33_pose + 21_left_hand + 21_right_hand) * 3_coords
NUM_CLASSES = 2
EPOCHS = 50
BATCH_SIZE = 16


# --- Load Data ---
# --- Load Data Safely ---
def load_array(path, dtype=np.float32):
    arr = np.load(path, allow_pickle=True)
    return np.array(arr, dtype=dtype)

X_train = load_array(f'{DATA_PATH}/X_train_landmarks.npy')
y_train = load_array(f'{DATA_PATH}/y_train.npy', dtype=np.int64)
X_val = load_array(f'{DATA_PATH}/X_val_landmarks.npy')
y_val = load_array(f'{DATA_PATH}/y_val.npy', dtype=np.int64)
X_test = load_array(f'{DATA_PATH}/X_test_landmarks.npy')
y_test = load_array(f'{DATA_PATH}/y_test.npy', dtype=np.int64)

print(f"Data loaded: Train {X_train.shape}, Val {X_val.shape}, Test {X_test.shape}")
# X_train = np.load(f'{DATA_PATH}/X_train_landmarks.npy')
# y_train = np.load(f'{DATA_PATH}/y_train.npy')
# X_val = np.load(f'{DATA_PATH}/X_val_landmarks.npy')
# y_val = np.load(f'{DATA_PATH}/y_val.npy')
# X_test = np.load(f'{DATA_PATH}/X_test_landmarks.npy')
# y_test = np.load(f'{DATA_PATH}/y_test.npy')

# print(f"Data loaded: Train {X_train.shape}, Val {X_val.shape}, Test {X_test.shape}")

# --- Build Model ---
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(SEQUENCE_LENGTH, NUM_FEATURES)),
    Dropout(0.5),
    LSTM(128, return_sequences=False),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# --- Callbacks ---
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(MODEL_SAVE_PATH, save_best_only=True, monitor='val_loss')

# --- Train Model ---
history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, model_checkpoint]
)

# --- Evaluate Model ---
print("\nEvaluating on test data...")
model.load_weights(MODEL_SAVE_PATH) # Load the best model
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['non_punch', 'punch']))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8909 - loss: 0.2849 - val_accuracy: 0.9388 - val_loss: 0.1634
Epoch 10/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8819 - loss: 0.3360 - val_accuracy: 0.9184 - val_loss: 0.1791
Epoch 11/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8883 - loss: 0.2889 - val_accuracy: 0.9388 - val_loss: 0.1780
Epoch 12/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8829 - loss: 0.2695

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8842 - loss: 0.2676 - val_accuracy: 0.9388 - val_loss: 0.1462
Epoch 13/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9244 - loss: 0.1977 - val_accuracy: 0.9184 - val_loss: 0.1687
Epoch 14/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9155 - loss: 0.1876

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9142 - loss: 0.1900 - val_accuracy: 0.9388 - val_loss: 0.1446
Epoch 15/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8956 - loss: 0.2274 - val_accuracy: 0.9184 - val_loss: 0.1689
Epoch 16/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9370 - loss: 0.1989 - val_accuracy: 0.8980 - val_loss: 0.2031
Epoch 17/50
14/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8963 - loss: 0.2354

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8934 - loss: 0.2388 - val_accuracy: 0.9388 - val_loss: 0.1444
Epoch 18/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9065 - loss: 0.2493 - val_accuracy: 0.8776 - val_loss: 0.2126
Epoch 19/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8993 - loss: 0.2873 - val_accuracy: 0.9388 - val_loss: 0.1522
Epoch 20/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9136 - loss: 0.2268 - val_accuracy: 0.9388 - val_loss: 0.1550
Epoch 21/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9046 - loss: 0.2138 - val_accuracy: 0.8776 - val_loss: 0.3037
Epoch 22/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8593 - loss: 0.2829 - val_accuracy: 0.9184 - val_loss: 0.1961
Epoch 23/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9172 - loss: 0.2117 - val_accuracy: 0.9184 - val_loss: 0.1586
Epoch 24/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9234 - loss: 0.1946 - val_accuracy: 0.9388 - val_l

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9192 - loss: 0.2148 - val_accuracy: 0.9592 - val_loss: 0.1432
Epoch 27/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9341 - loss: 0.1793 - val_accuracy: 0.8776 - val_loss: 0.3665
Epoch 28/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8966 - loss: 0.1893 - val_accuracy: 0.8776 - val_loss: 0.3467
Epoch 29/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8925 - loss: 0.2521 - val_accuracy: 0.8163 - val_loss: 0.3354
Epoch 30/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8562 - loss: 0.2899 - val_accuracy: 0.8980 - val_loss: 0.2127
Epoch 31/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8978 - loss: 0.2086 - val_accuracy: 0.8980 - val_loss: 0.2432
Epoch 32/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9276 - loss: 0.1798 - val_accuracy: 0.9184 - val_loss: 0.2125
Epoch 33/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9492 - loss: 0.1237 - val_accuracy: 0.9388 - val_l

In [ ]:
# -*- coding: utf-8 -*-
# SCRIPT 3: MOBILENETV2 + LSTM TRAINING

import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import TimeDistributed, LSTM, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
from google.colab import drive


# --- Parameters ---
DATA_PATH = '/content/drive/MyDrive/action_data_processed'
MODEL_SAVE_PATH = '/content/drive/MyDrive/mobilenet_lstm_model.h5'
SEQUENCE_LENGTH = 30
IMG_SIZE = (224, 224)
NUM_CLASSES = 2
EPOCHS = 50
BATCH_SIZE = 8

# --- Load Data ---
X_train = np.load(f'{DATA_PATH}/X_train_frames.npy')
y_train = np.load(f'{DATA_PATH}/y_train.npy')
X_val = np.load(f'{DATA_PATH}/X_val_frames.npy')
y_val = np.load(f'{DATA_PATH}/y_val.npy')
X_test = np.load(f'{DATA_PATH}/X_test_frames.npy')
y_test = np.load(f'{DATA_PATH}/y_test.npy')

print(f"Data loaded: Train {X_train.shape}, Val {X_val.shape}, Test {X_test.shape}")

# --- Preprocess Data ---
# Apply MobileNetV2-specific preprocessing
X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)
X_test = preprocess_input(X_test)
print("Data preprocessing complete.")

# --- Build Model ---
input_shape = (SEQUENCE_LENGTH, IMG_SIZE[0], IMG_SIZE[1], 3)
inputs = Input(shape=input_shape)

base_cnn = MobileNetV2(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,))
base_cnn.trainable = False  # Freeze the CNN base

x = TimeDistributed(base_cnn)(inputs)
x = TimeDistributed(GlobalAveragePooling2D())(x)
x = LSTM(64, return_sequences=True)(x)
x = Dropout(0.5)(x)
x = LSTM(128)(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# --- Callbacks ---
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(MODEL_SAVE_PATH, save_best_only=True, monitor='val_loss')

# --- Train Model ---
history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, model_checkpoint]
)

# --- Evaluate Model ---
print("\nEvaluating on test data...")
model.load_weights(MODEL_SAVE_PATH) # Load the best model
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['non_punch', 'punch']))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Data loaded: Train (228, 30, 224, 224, 3), Val (49, 30, 224, 224, 3), Test (50, 30, 224, 224, 3)
Data preprocessing complete.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 30, 224, 224,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 30, 7, 7, 1280) │     2,257,984 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 30, 1280)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 30, 64)         │       344,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 30, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,709,506 (10.34 MB)

 Trainable params: 451,522 (1.72 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
# -*- coding: utf-8 -*-
# SCRIPT 4: VGG19 + CONVLSTM TRAINING

import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import TimeDistributed, ConvLSTM2D, Dense, Dropout, Flatten, Multiply, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
from google.colab import drive

# --- Parameters ---
DATA_PATH = '/content/drive/MyDrive/action_data_processed'
MODEL_SAVE_PATH = '/content/drive/MyDrive/vgg19_convlstm_model.h5'
SEQUENCE_LENGTH = 30
IMG_SIZE = (224, 224)
NUM_CLASSES = 2
EPOCHS = 50
BATCH_SIZE = 4 # Smaller batch size due to memory usage of VGG19 and ConvLSTM
LEARNING_RATE = 1e-5
L2_REG = 0.001
DROPOUT_RATE = 0.5

# --- Load Data ---
# Note: For this model, we'll convert labels to float for binary_crossentropy
X_train = np.load(f'{DATA_PATH}/X_train_frames.npy')
y_train = np.load(f'{DATA_PATH}/y_train.npy').astype(float)
X_val = np.load(f'{DATA_PATH}/X_val_frames.npy')
y_val = np.load(f'{DATA_PATH}/y_val.npy').astype(float)
X_test = np.load(f'{DATA_PATH}/X_test_frames.npy')
y_test = np.load(f'{DATA_PATH}/y_test.npy').astype(float)

print(f"Data loaded: Train {X_train.shape}, Val {X_val.shape}, Test {X_test.shape}")

# --- Preprocess Data ---
# Normalize pixel values to [0, 1]
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
print("Data preprocessing complete.")

# --- Build Model ---
input_layer = Input(shape=(SEQUENCE_LENGTH, IMG_SIZE[0], IMG_SIZE[1], 3))
vgg = VGG19(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
vgg.trainable = False

x = TimeDistributed(vgg)(input_layer)
x = ConvLSTM2D(128, (3, 3), padding='same', kernel_regularizer=l2(L2_REG), return_sequences=True)(x)

# Simple Attention
attention = TimeDistributed(Dense(1, activation='tanh'))(x)
attention = tf.keras.layers.Softmax(axis=1)(attention)
x = Multiply()([x, attention])

x = TimeDistributed(Flatten())(x)
x = GlobalAveragePooling1D()(x)
x = Dense(64, activation='relu', kernel_regularizer=l2(L2_REG))(x)
x = Dropout(DROPOUT_RATE)(x)
# Use 1 neuron with sigmoid for binary classification
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_layer, outputs=output)
optimizer = Adam(learning_rate=LEARNING_RATE)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.summary()

# --- Callbacks ---
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(MODEL_SAVE_PATH, save_best_only=True, monitor='val_loss')
reduce_lr = ReduceLROnPlateau(factor=0.5, patience=5)

# --- Train Model ---
history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, model_checkpoint, reduce_lr]
)

# --- Evaluate Model ---
print("\nEvaluating on test data...")
model.load_weights(MODEL_SAVE_PATH)
y_pred_prob = model.predict(X_test).flatten()
y_pred = (y_pred_prob > 0.5).astype(int)

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['non_punch', 'punch']))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
# -*- coding: utf-8 -*-
# SCRIPT 5: SLOWFAST R50 FINE-TUNING (PYTORCH)

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import Compose, Lambda, Normalize, CenterCrop
from pytorchvideo.transforms import ApplyTransformToKey, ShortSideScale, UniformTemporalSubsample
from pytorchvideo.models.hub import slowfast_r50
from sklearn.metrics import classification_report, confusion_matrix
from google.colab import drive

# --- Parameters ---
DATA_PATH = '/content/drive/MyDrive/action_data_processed'
MODEL_SAVE_PATH = '/content/drive/MyDrive/slowfast_finetuned_model.pth'
SEQUENCE_LENGTH = 30 # Must match data prep
IMG_SIZE = 224
NUM_CLASSES = 2
EPOCHS = 25 # Fewer epochs for fine-tuning
BATCH_SIZE = 4 # Keep small for large models
LEARNING_RATE = 1e-4

# --- Device Configuration ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Load Data ---
X_train_np = np.load(f'{DATA_PATH}/X_train_frames.npy')
y_train_np = np.load(f'{DATA_PATH}/y_train.npy')
X_val_np = np.load(f'{DATA_PATH}/X_val_frames.npy')
y_val_np = np.load(f'{DATA_PATH}/y_val.npy')
X_test_np = np.load(f'{DATA_PATH}/X_test_frames.npy')
y_test_np = np.load(f'{DATA_PATH}/y_test.npy')

# --- Convert to PyTorch Tensors (and correct dimension order) ---
# PyTorch expects (N, T, C, H, W)
X_train = torch.from_numpy(X_train_np).permute(0, 1, 4, 2, 3).float()
y_train = torch.from_numpy(y_train_np).long()
X_val = torch.from_numpy(X_val_np).permute(0, 1, 4, 2, 3).float()
y_val = torch.from_numpy(y_val_np).long()
X_test = torch.from_numpy(X_test_np).permute(0, 1, 4, 2, 3).float()
y_test = torch.from_numpy(y_test_np).long()

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
print("Data loaded and converted to PyTorch DataLoaders.")

# --- Define Preprocessing Transforms for SlowFast ---
# This transform will be applied to each batch during training
transform = ApplyTransformToKey(
    key="video",
    transform=Compose([
        Lambda(lambda x: x / 255.0),
        Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
    ])
)

# --- Build Model ---
model = slowfast_r50(pretrained=True)
# Freeze all layers first
for param in model.parameters():
    param.requires_grad = False
# Replace the final classification head
model.blocks[6].proj = nn.Linear(in_features=2304, out_features=NUM_CLASSES)
model = model.to(device)

# --- Optimizer and Loss Function ---
# Only optimize the parameters of the new classification head
optimizer = optim.Adam(model.blocks[6].proj.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

# --- Training Loop ---
best_val_loss = float('inf')
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for videos, labels in train_loader:
        videos = videos.to(device)
        labels = labels.to(device)

        # Preprocess the batch
        batch = {"video": videos, "audio": None} # audio is a dummy key
        batch = transform(batch)
        videos = batch["video"]

        # SlowFast expects a list of two tensors: [slow_path, fast_path]
        fast_pathway = videos
        slow_pathway = videos[:, :, ::4, :, :] # Sample every 4th frame for slow path
        inputs = [slow_pathway, fast_pathway]

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # --- Validation Loop ---
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for videos, labels in val_loader:
            videos = videos.to(device)
            labels = labels.to(device)
            batch = {"video": videos, "audio": None}
            batch = transform(batch)
            videos = batch["video"]
            fast_pathway = videos
            slow_pathway = videos[:, :, ::4, :, :]
            inputs = [slow_pathway, fast_pathway]
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print("Model saved.")

# --- Evaluation ---
print("\nEvaluating on test data...")
model.load_state_dict(torch.load(MODEL_SAVE_PATH))
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for videos, labels in test_loader:
        videos = videos.to(device)
        batch = {"video": videos, "audio": None}
        batch = transform(batch)
        videos = batch["video"]
        fast_pathway = videos
        slow_pathway = videos[:, :, ::4, :, :]
        inputs = [slow_pathway, fast_pathway]
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=['non_punch', 'punch']))
print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))